# Building workspaces for calibration
In this notebook, we will take all the measurements we pulled from EXFOR and build `ElasticWorkspace` objects defined in this repo. These classes allow us to precompute useful things (like kinematics and Coulomb scattering amplitudes) for model calculations for each measurement, so that we can toss in a model and a bunch of parameters and quickly generate and ensemble of predictions for our data corpus. and calculate things like $\chi^2$ distributions and empirical coverages.

In [1]:
import pickle
from collections import OrderedDict

import numpy as np
from periodictable import elements
from tqdm import tqdm

In [2]:
from exfor_tools import curate, quantities, reaction

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [3]:
import elm

## Read in the measurements

In [4]:
with open("nn_elastic_data.pkl", "rb") as f:
    nn_data = pickle.load(f)

In [5]:
with open("pp_elastic_data.pkl", "rb") as f:
    pp_data = pickle.load(f)

In [6]:
all_data = list(pp_data.values()) + list(nn_data.values())
all_data_by_entry, sys_uncertainties_by_entry = (
    curate.cross_reference_entry_systematic_err(all_data)
)

### If we want to only use a subset of targets we can do something like this:

In [7]:
#nn_data = {target: nn_data[target] for target in ((48, 20), (40, 20))}
#pp_data = {target: pp_data[target] for target in ((48, 20), (40, 20))}

### first, let's do $(n,n)$

In [8]:
%%time
nn_elastic_workspaces = elm.corpus.build_workspaces_from_data("dXS/dA", nn_data)

CPU times: user 1min 7s, sys: 114 ms, total: 1min 7s
Wall time: 1min 7s


In [9]:
with open("nn_elastic_workspaces.pkl", "wb") as f:
    pickle.dump(nn_elastic_workspaces, f)

### next, let's do $(p,p)$

In [10]:
%%time
pp_elastic_workspaces_abs = elm.corpus.build_workspaces_from_data("dXS/dA", pp_data) 

CPU times: user 1min 55s, sys: 164 ms, total: 1min 55s
Wall time: 1min 55s


In [11]:
%%time
pp_elastic_workspaces_ruth = elm.corpus.build_workspaces_from_data("dXS/dRuth", pp_data)

CPU times: user 1min 8s, sys: 115 ms, total: 1min 8s
Wall time: 1min 8s


In [12]:
pp_elastic_workspaces = pp_elastic_workspaces_abs + pp_elastic_workspaces_ruth

In [13]:
with open("pp_elastic_workspaces.pkl", "wb") as f:
    pickle.dump(pp_elastic_workspaces, f)